In [ ]:
import uproot
import awkward as ak

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

from scipy.optimize import curve_fit

plt.rc("pdf", fonttype=42)
plt.rc("pdf", fonttype=42)
plt.rc("axes", labelsize=20)
plt.rc("xtick", labelsize=18, top=True, direction="in")
plt.rc("ytick", labelsize=18, right=True, direction="in")
plt.rc("axes", titlesize=20)
plt.rc("legend", fontsize=18)

import os
curdir = os.getcwd()
while not os.path.exists(".projectroot") and os.getcwd() != os.path.expanduser('~'):
    os.chdir("..")
try:
    proot = open(".projectroot").readline()
except:
    print("Could not find file '.projectroot'.")
    print("Fallback: Look for the file 'rpad.py' in the current working directory (%s)." % curdir)
    print("Fonts in figures might not appear as intended, either.")
os.chdir(curdir)
import sys
import matplotlib.font_manager as font_manager
if len(proot) > 0:
    sys.path.append(proot+"/python")
    font_dir = proot + '/fonts'
    font_files = font_manager.findSystemFonts(font_dir)
    for f in font_files:
        font_manager.FontManager.addfont(font_manager.fontManager, path=f)
    plt.rcParams.update({"font.family": "serif", "font.serif": "Linux Libertine O", "font.cursive": "Linux Libertine O", "font.sans-serif": "Linux Libertine O", "mathtext.fontset": "custom"})
from rpad import rpad

os.makedirs("tmp", exist_ok=True)

dssds = ["U3", "U4", "U5"]
ids = [1, 2, 3]

In [ ]:
# v, Ep, fEdep, bEdep = np.loadtxt('Edep_vs_Ep_U3_200keV.dat', unpack=True)
v, Ep, fEdep, bEdep = np.loadtxt('Edep_vs_Ep_U3.dat', unpack=True)
Ep /= 1e3
fEdep /= 1e3
bEdep /= 1e3
v = np.rad2deg(v)

In [ ]:
v0 = np.min(v)
v1 = np.max(v)

Ep0    =    Ep[(v == v0) & (Ep <= 8.)]
fEdep0 = fEdep[(v == v0) & (Ep <= 8.)]
bEdep0 = bEdep[(v == v0) & (Ep <= 8.)]

Ep1    =    Ep[(v == v1) & (Ep <= 8.)]
fEdep1 = fEdep[(v == v1) & (Ep <= 8.)]
bEdep1 = bEdep[(v == v1) & (Ep <= 8.)]

In [ ]:
#plt.figure(figsize=(1.3*7, 1.3*4))
plt.plot(Ep0, fEdep0, 'k-', lw=2.6,  label='$\Delta E\,\,\mathrm{detector},\,\\theta =%d°$' % 0, zorder=10)
plt.plot(Ep0, bEdep0, 'k--', lw=2.6, label='$E\,\,\mathrm{detector},\,\\theta =%d°$' % 0, zorder=11)
plt.plot(Ep1, fEdep1, ls='-', c='grey', lw=1.6, label='$\Delta E\,\,\mathrm{detector},\,\\theta =%d°$' % round(v1))
plt.plot(Ep1, bEdep1, ls='--', c='grey', lw=1.6, label='$E\,\,\mathrm{detector},\,\\theta =%d°$' % round(v1))
plt.vlines(Ep0[fEdep0 == np.max(fEdep0)], np.min(bEdep0), np.max(fEdep0), color='k', lw=0.7, zorder=1)
plt.vlines(Ep0[np.nonzero(bEdep0)[0][0]], bEdep0[np.nonzero(bEdep0)[0][0]] - 0.1, fEdep0[np.nonzero(bEdep0)[0][0]], color='k', lw=0.7, zorder=1)
plt.vlines(Ep1[fEdep1 == np.max(fEdep1)], np.min(bEdep0), np.max(fEdep1), color='grey', lw=0.7, zorder=1)
plt.vlines(Ep0[np.nonzero(bEdep1)[0][0]], bEdep1[np.nonzero(bEdep1)[0][0]], fEdep1[np.nonzero(bEdep1)[0][0]], color='grey', lw=0.7, zorder=1)
plt.hlines(np.max(fEdep0), 1.2, Ep0[fEdep0 == np.max(fEdep0)], color='k', lw=0.7, ls='--', zorder=1)
plt.hlines(fEdep0[np.nonzero(bEdep0)[0][0]], 1.2, Ep0[np.nonzero(bEdep0)[0][0]], color='k', lw=0.7, ls='--', zorder=1)
plt.hlines(np.max(fEdep1), 1.2, Ep1[fEdep1 == np.max(fEdep1)], color='grey', lw=0.7, ls='--', zorder=1)
plt.hlines(fEdep1[np.nonzero(bEdep1)[0][0]], 1.2, Ep1[np.nonzero(bEdep1)[0][0]], color='grey', lw=0.7, ls='--', zorder=1)
plt.xlabel("$E_{\mathrm{p}}$ (MeV)")
plt.ylabel("$E_{\mathrm{dep}}$ (MeV)")
plt.xticks(range(0, 7))
plt.xticks(np.arange(0, 7, 0.2), minor=True)
plt.yticks(range(0, 7))
plt.yticks(np.arange(0, 7, 0.5), minor=True)
plt.xlim(-0.2, 5.5)
plt.ylim(-0.2, 6.5)
plt.legend()
plt.savefig('Edep_vs_Ep.pdf', dpi=300, bbox_inches='tight')
plt.show()
print("lower: [", Ep0[fEdep0 == np.max(fEdep0)][0], ",", Ep0[np.nonzero(bEdep0)[0][0]], "]")
print("upper: [", Ep1[fEdep1 == np.max(fEdep1)][0], ",", Ep0[np.nonzero(bEdep1)[0][0]], "]")

In [ ]:
f = uproot.open(proot+"/data/bananas2/all.root")
t = f["a"]

v0 = np.deg2rad(2)
v1 = np.deg2rad(5)
v2 = np.deg2rad(33)
v3 = np.deg2rad(39)

b1 = t.arrays(["fEdep", "bEdep"], "(id == 1) & (%f <= angle) & (angle <= %f)" % (v0, v1))
b2 = t.arrays(["fEdep", "bEdep"], "(id == 1) & (%f <= angle) & (angle <= %f)" % (v2, v3))
fEdeplow, bEdeplow = rpad(b1.fEdep/1e3, b1.bEdep/1e3)
fEdepupp, bEdepupp = rpad(b2.fEdep/1e3, b2.bEdep/1e3)

In [ ]:
print(len(fEdeplow[~np.isnan(fEdeplow)]))
print(len(fEdepupp[~np.isnan(fEdepupp)]))

In [ ]:
scale = 1.3

In [ ]:
plt.rc("pdf", fonttype=42)
plt.rc("pdf", fonttype=42)
plt.rc("axes", labelsize=scale*20)
plt.rc("xtick", labelsize=scale*18, top=True, direction="in")
plt.rc("ytick", labelsize=scale*18, right=True, direction="in")
plt.rc("axes", titlesize=scale*20)
plt.rc("legend", fontsize=18)

In [ ]:
plt.figure(figsize=(6.4*scale, 4.8*scale))

plt.hlines(fEdep0[bEdep0 > 0][0], -0.2, 2, ls='-', color='k', lw=0.5)
plt.hlines(fEdep1[bEdep1 > 0][0], -0.2, 2, ls='-', color='grey', lw=0.5)

plt.fill_between([0, bEdep0[bEdep0 > 0][0]*3], 0, np.max(fEdep0), fc='none', ec='k', lw=2, label='$\Delta E$-contained events$,\,\\theta =%d°$' % round(np.rad2deg(v0)), zorder=10)
plt.plot(bEdep0[bEdep0 > 0], fEdep0[bEdep0 > 0], 'k--', lw=3, label='Telescope events$,\,\\theta =%d°$' % round(np.rad2deg(v0)))

plt.plot(-1, -1, '.', c='k', ms=12, label='Data for $%d° \leq \\theta \leq %d°$' % (round(np.rad2deg(v0)), round(np.rad2deg(v1))))

plt.fill_between([0, bEdep1[bEdep1 > 0][0]*3], 0, np.max(fEdep1), fc='none', ec='grey', lw=2, label='$\Delta E$-contained events$,\,\\theta =%d°$' % round(np.rad2deg(v3)), zorder=9)
plt.plot(bEdep1[bEdep1 > 0], fEdep1[bEdep1 > 0], ls='--', c='grey', lw=3, label='Telescope events$,\,\\theta =%d°$' % round(np.rad2deg(v3)))

plt.plot(-1, -1, '.', c='grey', ms=12, label='Data for $%d° \leq \\theta \leq %d°$' % (round(np.rad2deg(v2)), round(np.rad2deg(v3))))

plt.plot(bEdeplow, fEdeplow, '.', c='k', ms=2, zorder=12, alpha=1)
plt.plot(bEdepupp, fEdepupp, '.', c='grey', ms=2, zorder=11, alpha=1)

plt.fill_between(bEdep0[bEdep0])

plt.xticks(np.arange(0, 8, 1))
plt.xticks(np.arange(-0.2, 8, 0.2), minor=True)
plt.yticks(np.arange(0, 4, 1))
plt.yticks(np.arange(-0.5, 3.5, 0.1), minor=True)
plt.ylim(-0.1, 3.1)
plt.xlim(-0.2, 7.8)
plt.xlabel("$E_{\mathrm{dep}}^{E}$ (MeV)")
plt.ylabel("$E_{\mathrm{dep}}^{\Delta E}$ (MeV)")
leg = plt.legend(framealpha=1, bbox_to_anchor=(0.5, 0.45))
leg.set_zorder(20)
plt.savefig('Edep_vs_Edep.pdf', bbox_inches='tight')
plt.show()